# Sky model tutorial

Most quantities in the sky model interface is `astropy.unit.Quantity` objects

In [1]:
import astropy.units as u

## Loading a sky model

There are two ways to initialize the Cosmoglobe Sky Model. The default way is to use the ``cosmoglobe.skymodel`` function, which by default downloads and cache a HDF5 file containing the model data (for a specific NSIDE).

Alternatively, we can use ``cosmoglobe.model_from_chain`` if we have direct access to a commander3 chain.

### Using ``cosmoglobe.skymodel`` (currently unavailable)

This method is currently unavailable as we are working on producing the first stable data release. As a result the below code will not function as of now.

In [2]:
# from cosmoglobe import skymodel
# model = skymodel(nside=256)

### Using ``cosmoglobe.model_from_chain``

This method initializes a model directly from a commander3 HDF5 chainfile.

In [3]:
from cosmoglobe import model_from_chain

# Path to commander3 chainfile
chain = "/Users/metinsan/Documents/doktor/Cosmoglobe_test_data/chain_test.h5"

# Initializing the Cosmoglobe Sky Model at NSIDE 256
model = model_from_chain(chain, nside=256)

ImportError: cannot import name 'model_from_chain' from 'cosmoglobe' (/opt/anaconda3/envs/cosmoglobe_test/lib/python3.8/site-packages/cosmoglobe/__init__.py)

## The model object 

To get an overview of the model and the sky components in it, we can print the model object:

In [ ]:
model

We can disable any component in the model with the `model.disable()` function:

In [ ]:
model.disable('radio')
model

And re-enable them using the `model.enable` function:

In [ ]:
model.enable('radio')

## Model components

Let us explore the sky components in further detail. Each component can be individually accessed through the attribute names seen in the ``print(model)`` output.

In [ ]:
print(model.dust, model.synch) # Etc..

### Component attributes

The model data is stored in the following component attributes:

- `amp`: Amplitude map at the reference frequency (Commander average posterior map)

- `freq_ref`: Reference frequency of `amp`

- `spectral_parameters`: A dictionary containing the spectral parameters

We can print these attributes:

In [ ]:
print(model.dust.amp)
print(model.dust.freq_ref)
print(model.dust.spectral_parameters)

### Visualizing component

We can visualize the maps of a component (`amp` or a spectral parameter map) using `healpy.mollview`. 

Alternatively, we can use `cosmoglobe.plot`, which is a wrapper on healpy's mollview function that features built-in esthetique choices. 

In this tutorial we will use `cosmoglobe.plot`. For a more in in-depth overview of the built-in plot function, please see the Plotting tutorial.

In [ ]:
from cosmoglobe import plot

Lets plot some of the reference amplitude maps that the model uses:

In [ ]:
dust_amp_I = model.dust.amp[0]  # Stokes I map of dust

plot(
    dust_amp_I,
    title='Dust Stokes I',
    min=30,
    max=3000,
    norm='log',
    cmap='sunburst',
)

In [ ]:
dust_amp_Q = model.dust.amp[1]  # Stokes Q map of dust

plot(
    dust_amp_Q,
    title='Dust Stokes Q',
    min=-100,
    max=100,
    norm='log',
    cmap='iceburn',
)

## Simulations

The primary use case of the cosmoglobe software is to provide the community with astrophysical maps generated with the Cosmoglobe Sky Model through commander. The mean posterior maps (given at some reference frequency) are directly accessible through component attributes of the model object as demonstrated in the above section.

These maps can additionally be extrapolated to an arbitrary frequency to produce simulations of the sky. In the following we look at how we generate simulations with the cosmoglobe software:

### Simulating component emission

We can simulate the emission from a component at an arbitrary frequency `freq` by calling the component's `__call__` method, e.g `model.dust(freq)`.

This function takes in the following key word arguments:

- `freqs` : A frequency, or a list of frequencies for which to evaluate the sky emission (must be astropy quantities).

- `bandpass` : Bandpass profile corresponding to the frequencies (optional). 

- `fwhm` : The full width half max parameter of the Gaussian used to smooth the output (optional).

- `output_unit` : The output units of the emission (By default the output unit of the model is always in uK_RJ

Below, we illustrate a simulation of synchrotron emission at $20\;\mathrm{GHz}$:

In [ ]:
# Simulated synchrotron emission at 20GHz
simulated_emission = model.synch(20*u.GHz)

plot(
    simulated_emission[0],
    title='Synchrotron simulated at 20 GHz', 
    cmap='swamp',
)

We will also demonstrate the other optional keywords. Here we simulate free-free emission at $20\;\mathrm{GHz}$ seen with a beam with a FWHM of $30\;'$ with output units set to $\mathrm{MJy/sr}$:

In [ ]:
# Simulated free-free emission at 60GHz seen by a 30 arcmin beam in units of MJy/sr
simulated_emission = model.ff(60*u.GHz, fwhm=30*u.arcmin, output_unit='MJy/sr')

plot(
    simulated_emission[0], 
    title=f'Free-free simulated at 60GHz with a 30 arcmin beam', 
    unit=ff_60GHz,
    cmap='freeze',
)

### Simulating model emission

Similarly, by calling the model's `__call__` function (which takes in the same keyword arguments), we can simulate the sky emission over the full model at a given frequency:

In [ ]:
# Simulated full sky emission at 100GHz seen by a 60 arcmin beam in units of uK_RJ
emission = model(100*u.GHz, fwhm=60*u.arcmin)[0]

plot(
    emission, 
    title='full sky simulated at 100GHz with a 60 arcmin beam',  
    min=-3400,
    max=3400,
    norm='hist',
)


It is possible to remove the solar dipole from the model, by calling `model.cmb.remove_dipole()`:

In [ ]:
# Remove the solar dipole
model.cmb.remove_dipole()

# Simulated full sky emission at 100GHz seen by a 60 arcmin beam in units of uK_RJ
fullsky_emission_100GHz = model(100*u.GHz, fwhm=60*u.arcmin)[0]

plot(
    fullsky_emission_100GHz, 
    title='full sky simulated at 100GHz with a 60 arcmin beam', 
    min=-300,
    max=300,
)


### Bandpass integration

We can also make simulations that have integrated the sky emission over a given bandpass. By default 
if only a list of frequencies are supplied without a explicit bandpass, a top-hat bandpass will be assumed.

In [ ]:
import numpy as np

frequencies = np.arange(100, 110, 50)*u.GHz

emission = model(frequencies, fwhm=40*u.arcmin)

plot(
    emission, 
    title='Sky simulated with a top-hat bandpass', 
    norm='hist',
)

In the following we use the WMAP K-band bandpass profile:

In [ ]:
bandpass = "/Users/metinsan/Documents/doktor/Cosmoglobe_test_data/wmap_bandpass.txt"
frequencies, bandpass, _ = np.loadtxt(bandpass, unpack=True)

#add astropy units to the bandpass
frequencies*= u.GHz
bandpass *= u.K

In [ ]:
# Note: Solar dipole is removed from the model
wmap_kband_emission = model(frequencies, bandpass, fwhm=0.88*u.deg, output_unit='mK')[0]

plot(
    wmap_kband_emission, 
    title='Sky as seen by the WMAP K-band', 
    unit=wmap_kband_emission.unit,
    norm='hist',
)

## Point sources

The radio component does not internally store amplitude maps, but rather just a single amplitude value estimated in commander per point source. Each source is mapped to a healpix map with a gaussian beam whenever the model (or radio component) is called.

In [ ]:
# Point sources seen at 40GHz over a 50 arcmin gaussian beam
emission = model.radio(30*u.GHz, fwhm=50*u.arcmin)[0]

plot(
    emission, 
    title='Point sources at 40 GHz with a 50 arcmin gaussian beam', 
    norm='log', 
    cmap='CMRmap'
)